In [33]:
import requests
from datetime import date,timedelta,datetime
from bs4 import BeautifulSoup as Bs
import re

In [182]:
target = {
    'bpom':'https://www.pom.go.id/new/browse/more/pers/',
    'fssc':'https://www.fssc.com/insights/?types=update',
    'gfsi':'https://mygfsi.com/',
    'ec': 'https://food.ec.europa.eu/news_en',
    'usfda':'https://www.fda.gov/news-events/fda-newsroom/press-announcements',
    'codex':'https://www.fao.org/fao-who-codexalimentarius/news-and-events/en/',
}

In [308]:
indonesian_month_mapper = {
    'Januari':'January',
    'Februari':'February',
    'Maret':'March',
    'April':'April',
    'Mei':'May',
    'Juni':'June',
    'Agustus':'August',
    'September':'September',
    'Oktober' :'October',
    'November':'November',
    'Desember':'December'
}

#### 1.Extract news from  BPOM website  

In [2]:
today = date.today() # used for All other target Website 

In [3]:
last_week = today - timedelta(days=7) # used for All other target Website 

In [4]:
str_today = today.strftime("%d-%m-%Y")

In [5]:
str_today
str_last_week = last_week.strftime("%d-%m-%Y")

In [328]:
req = requests.get(f'https://www.pom.go.id/new/browse/more/pers/{str_last_week}/{str_today}')


In [329]:
bs = Bs(req.content,parser='lxml')

In [330]:
selected_entry_news = bs.select('h4.entry-title a')
selected_entry_date = bs.select('p.e_meta a span')

In [316]:
bpom_news = {}
if len(selected_entry_date) and (selected_entry_news) == 0 : 
    str_today = today.strftime("%d %B %Y")
    bpom_news[str_today] = "No update this week "
    

In [332]:
for news in selected_entry_news : 
    print(news.getText())

BPOM Bangun Sinergi Wujudkan Kemandirian Produk Darah Dalam Negeri
BPOM Terbitkan EUA Booster Heterolog Vaksin Inavac


In [320]:
list_news_bpom=[]
for s in selected_entry_news: 
    list_news_bpom.append(s.getText())

#### 2. Extract News From FSSC Foundation Website 

In [13]:
url = "https://www.fssc.com/insights/?types=update"

In [14]:
req = requests.get(url)
bs = Bs(req.content,'lxml')

In [16]:
dates = bs.select("div.insight-events-header p") 
titles = bs.select("h4.insight-events-title")

In [29]:
list_date = []
for d in dates :
    list_date.append(d.getText())

list_title = []
for t in titles : 
    list_title.append(t.getText())


In [309]:
def check_zero_pad_date(fs,str_date): 
    if fs == "usfda":             
        split = re.split(' |, ',str_date)
     
        if len(split[1]) == 1 :
            split[1] = "0" + split[1]
           
            return " ".join(split)
        else : 
           
           
            return str_date
    elif fs == "fssc": 
        split = re.split(' ',str_date)
        if len(split[0]==1):
            split[0] = "0"+split[0]
            return " ".join(split)
        else : 
            return str_date
    elif fs == 'bpom': 
        split = re.split(' ',str_date)
        split[1] = indonesian_month_mapper[split[1]]
        if len(split[0]) == 1 : 
            split[0] = "0"+split[0]
            return " ".join(split)
        else : 
            return " ".join(split)
 


In [322]:
def check_date_within_a_week(fs,d,last_week_date):
    try : 
        if fs == 'fssc': 
            print("agus1")
            matched = re.findall('\d+ .+, \d{4}',d)
            new_str = check_zero_pad_date(fs,matched[0])
            date_object = datetime.strptime(new_str,"%d %B, %Y")
           
        elif (fs == 'gfsi' or fs == 'codex' or fs == 'bpom'): 
            matched = re.findall('\d+ .+ \d{4}',d)
            if fs == 'gfsi': 
                date_object = datetime.strptime(matched[0],"%d %b %Y")
            elif fs == 'codex': 
                date_object = datetime.strptime(matched[0],"%d %B %Y")
            elif fs == 'bpom': 
                new_str = check_zero_pad_date(fs,matched[0])
                date_object = datetime.strptime(new_str,"%d %B %Y")
        elif fs == "usfda": 
            matched = re.findall('.+ \d+, \d{4}',d)
          
            new_str = check_zero_pad_date(fs,matched[0])
         
            date_object = datetime.strptime(new_str,"%B %d, %Y")
      
        if date_object.date() >= last_week_date : 
            str_date = date_object.date().strftime("%d %B %Y")
  
            return str_date
    except:
        print("errrrooorrrrr")
        # if matched: 
        #     new_date_string = '0'+matched[0] # to handle non-zero padded date 
        #     date_object = datetime.strptime(new_date_string,"%d %B, %Y")
        #     if date_object.date() > last_week_date : 
        #         str_date = date_object.date().strftime("%d %B %Y")
        #         return str_date

        # else : 
        #     pass
    
            



#### 3. Extract News From GFSI Website 

In [95]:
url = 'https://mygfsi.com/'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
# you need to add headers of user agent in this website. 
req = requests.get(url,headers=headers)
bs = Bs(req.content,'lxml')

In [122]:
featured_news = bs.select('div.featured-news.section.grey-bg .section-title a')
featured_dates = bs.select('div.featured-news.section.grey-bg .grid-nav .left')

In [150]:
list_news = [n.getText() for n in featured_news]
list_dates = [d.getText() for d in featured_dates]

In [153]:
print(list_dates)

['22 Nov 2022', '14 Nov 2022', '9 Nov 2022']


In [325]:
def create_news_dict(fs,list_news,list_dates,today_date,last_week_date):
     news_dict = {}
     for d,n in zip(list_dates,list_news): 
          valid_date = check_date_within_a_week(fs,d,last_week_date)
          print(valid_date)
          if valid_date : 
               news_dict[n] = valid_date
     if len(news_dict) == 0 : 
          str_today = today_date.strftime("%d %B %Y")
          news_dict["No update this week"] = str_today
     
     return news_dict
          


In [ ]:
news_dict_gfsi = create_news_dict('gfsi',list_news,list_dates,today,last_week)

In [176]:
news_dict_gfsi

{'22 November 2022': 'GFSI Science & Technology Advisory Group Publish Papers on Areas Set to Significantly Impact Food Safety'}

#### 4. Extract news from Codex Alimentarius 

In [185]:
req = requests.get('https://www.fao.org/fao-who-codexalimentarius/news-and-events/en/')
bs = Bs(req.content,'lxml')
titles = bs.select('h3.list-title a')
dates = bs.select('div.list-date')

In [190]:
list_titles = [x.getText() for x in titles]
list_dates = [x.getText() for x in dates]

In [209]:
new_dict_codex = create_news_dict('codex',list_titles,list_dates,today,last_week)

In [210]:

new_dict_codex

{'24 November 2022': 'New eLearning course: Master the foundations of Codex on any digital device',
 '23 November 2022': 'Codex votes but zilpaterol remains unresolved',
 '21 November 2022': 'CAC45 opens in FAO / QU Dongyu: science and data play a crucial role in setting standards',
 '19 November 2022': 'CAC45 / Steve Wearne outlines conduct of discussions on zilpaterol'}

#### 5. Extract News from US FDA 

In [247]:
req = requests.get('https://www.fda.gov/news-events/fda-newsroom/press-announcements')
bs = Bs(req.content,'lxml')
dates = bs.select('span.field-content a time')


In [251]:
news_tag = bs.select('span.field-content a')
news = []
for n in news_tag : 
    new_n = re.sub('.+ \d+, \d{4}\n - ','',n.getText())
    news.append(new_n)
    

In [273]:
list_dates = [x.getText() for x in dates]


In [291]:
news

['FDA Approves First Gene Therapy to Treat Adults with Hemophilia B',
 'FDA Roundup: November 22, 2022 ',
 'FDA Roundup: November 18, 2022 ',
 'FDA Approves First Drug That Can Delay Onset of Type 1 Diabetes',
 'FDA Warns Seven Companies for Selling Dietary Supplements with Claims to Treat Cardiovascular Disease',
 'FDA Spurs Innovation for Human Food from Animal Cell Culture Technology ',
 'FDA Warns Firms for Selling Illegal E-cigarettes That Look Like Toys, Food, and Cartoon Characters',
 'FDA Roundup: November 15, 2022 ',
 'FDA Advances the New Era of Smarter Food Safety, Finalizing Foundational Rule to Improve Traceability of Contaminated Foods',
 'FDA Conditionally Approves First Drug to Manage Acute Onset of Pancreatitis in Dogs']

In [ ]:
new_dict_usfda = create_news_dict('usfda',news,list_dates,today,last_week)


In [ ]:
new_dict_usfda